# Imports

In [1]:
!pip install -U clifford

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 5.4 MB/s eta 0:00:00


In [2]:
import h5py
import gdown
import numpy as np
from clifford import Cl

# Dataset

## Bifurcating dataset

In [9]:
bifurcating_dataset_file_id = '1oGqW1M2B6AY389ZKIvkDm8RpZcOZQdC0'
download_link = f'https://drive.google.com/uc?id={bifurcating_dataset_file_id}'
bifurcating_samples = gdown.download(download_link, quiet=False)

with h5py.File(bifurcating_samples, 'r') as hdf5_file:
  bifurcating_sample_0000_wss = hdf5_file['sample_0000']['wss'][()]  # NumPy array
  bifurcating_sample_0000_pressure = hdf5_file['sample_0000']['pressure'][()]
  bifurcating_sample_0000_pos = hdf5_file['sample_0000']['pos'][()]
  bifurcating_sample_0000_face = hdf5_file['sample_0000']['face'][()]
  bifurcating_sample_0000_inlet = hdf5_file['sample_0000']['inlet_idcs'][()]



Downloading...
From: https://drive.google.com/uc?id=1oGqW1M2B6AY389ZKIvkDm8RpZcOZQdC0
To: /content/database.hdf5
100%|██████████| 1.76G/1.76G [00:19<00:00, 92.8MB/s]


In [11]:
bifurcating_sample_0000_inlet

array([    2,     1,     0,     5,     8, 17421, 17422,     3, 11161,
       11162,     7,     6, 11191,  1809, 17423, 17424,    36,    35,
          34,    42,    45, 17425, 17426,    37, 17376, 17377,    44,
          43, 16820,    49, 17427, 17428,  1903,  1906, 17565,    55,
          54,    53,  3875,  3874, 17429, 17430,    56,  1919,  1920,
       11193, 11192, 17507,  1905,  1904, 16822, 16821, 17444, 17445,
       16963, 16964, 17446, 13647, 13645, 17447,  8348,  8347,  8346,
       17401, 17397, 17448, 17449,  8349, 15531, 15532,  8410,  8409,
        8408, 16322, 13610, 17450, 17451,  8411, 15491, 16540,  8436,
        8435,  8434, 17414, 17413, 17452, 17453,  8437, 16630, 17400,
       17375, 11154, 17411, 17412, 17601, 17506, 15745, 15744, 17508,
       14366, 14368, 14367, 13063, 14362, 17305, 17516, 16634, 16636,
       17558, 13646, 13648, 16649, 16156, 17514, 17515, 17415, 17113,
       17410, 14349, 17610, 17596, 14373, 14370, 17517, 14372, 14371,
       14378, 14381,

In [13]:
bifurcating_sample_0000_pressure.shape

(19217,)

# Single dataset

In [ ]:
single_dataset_file_id = '13yLm1UaTPHh7VlZlHtw4giCXp1jARHYd'
download_link = f'https://drive.google.com/uc?id={single_dataset_file_id}'
single_samples = gdown.download(download_link, quiet=False)

with h5py.File(single_samples, 'r') as hdf5_file:
  single_sample_0000_wss = hdf5_file['sample_0000']['wss'][()]  # NumPy array
  single_sample_0000 = hdf5_file['sample_0000'] # we cannot access the whole sample, but only its attributes (wss, pressure, pos, face, inlet_idcs) because of the hdf5 group rules



In [ ]:
single_sample_0000_wss

array([[ 3.76564560e+01, -1.58873138e+01,  1.03681274e-01],
       [ 3.31618729e+01, -1.44140673e+01, -2.68949002e-01],
       [ 3.33288002e+01, -1.47469664e+01,  3.13610099e-02],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)

In [ ]:
single_sample_0000_wss.shape

(10475, 3)

In [ ]:
single_sample_0000

<Closed HDF5 group>

# Multivectors

In [37]:
# sample 0000

layout, blades = Cl(3, 0, 1)

# wss as a scalar
wss_multivector = 0
for row in bifurcating_sample_0000_wss:
  vector = row[0] * blades[''] + row[1] * blades[''] + row[2] * blades['']
  wss_multivector += vector


# pressure as a line
pressure_multivector = 0
for elem in bifurcating_sample_0000_pressure:
  vector = elem * blades['e12']
  pressure_multivector += vector


# pos as a point
pos_multivector = 0
for row in bifurcating_sample_0000_pos:
  vector = row[0] * blades['e123'] + row[1] * blades['e124'] + row[2] * blades['e134']
  pos_multivector += vector


# face as a plane
face_multivector = 0
for row in bifurcating_sample_0000_face:
  vector = row[0] * blades['e1'] + row[1] * blades['e2'] + row[2] * blades['e3']
  face_multivector += vector


# inlet_idcs as a pseudoscalar
inlet_multivector = 0
for elem in bifurcating_sample_0000_inlet:
  vector = elem * blades['e1234']
  inlet_multivector += vector


# sample_0000 multivector
sample_0000_multivector = wss_multivector + pressure_multivector + pos_multivector + face_multivector + inlet_multivector

In [36]:
max(sample_0000_multivector.grades()) # if 4,  then the dimension is correct

4

In [ ]:
blades

{'': 1,
 'e1': (1^e1),
 'e2': (1^e2),
 'e3': (1^e3),
 'e4': (1^e4),
 'e12': (1^e12),
 'e13': (1^e13),
 'e14': (1^e14),
 'e23': (1^e23),
 'e24': (1^e24),
 'e34': (1^e34),
 'e123': (1^e123),
 'e124': (1^e124),
 'e134': (1^e134),
 'e234': (1^e234),
 'e1234': (1^e1234)}